In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0


In [3]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])


In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [5]:
model.fit(train_images, train_labels, epochs=100, batch_size=32)


Epoch 1/100
1875/1875 [==============================] - 18s 8ms/step - loss: 0.4962 - accuracy: 0.8249 0s - loss: 0
Epoch 2/100
1875/1875 [==============================] - 14s 7ms/step - loss: 0.3716 - accuracy: 0.8662 
Epoch 3/100
1875/1875 [==============================] - 15s 8ms/step - loss: 0.3363 - accuracy: 0.8774 1s - loss: 0.3381 - accuracy - ETA: 
Epoch 4/100
1875/1875 [==============================] - 14s 8ms/step - loss: 0.3102 - accuracy: 0.8862 1s - loss: - ETA: 0s - loss: 0.3100 - ac
Epoch 5/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2922 - accuracy: 0.8928
Epoch 6/100
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2773 - accuracy: 0.8975 0s - loss: 0.2784 - accu - ETA: 0s - loss: 0.2774 - accuracy
Epoch 7/100
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2663 - accuracy: 0.9008
Epoch 8/100
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2546 - accuracy: 0.9051
Epoch 9/100
187

In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


313/313 [==============================] - 4s 5ms/step - loss: 0.7872 - accuracy: 0.8854
Test accuracy: 0.8853999972343445


In [7]:
predictions = model.predict(test_images)
